In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, getpass
lari_id='fdb573a66f50' ## lari_ccblin008
#lari_id='19b775bb238f' ## lari_g5ub
#lari_id='133898b2b079' ## lari_jm
#lari_id='cb48a51bf9e5' ## lari_dub
#lari_id=''
if lari_id:
    print('Enter processing passcode for lari node {}'.format(lari_id))
    lari_passcode=getpass.getpass()
else:
    lari_passcode=''
os.environ['LARI_ID']=lari_id
os.environ['LARI_PASSCODE']=lari_passcode

Enter processing passcode for lari node fdb573a66f50


In [3]:
import os
import sys
dir0 = os.path.split(os.getcwd())[0]
if dir0 not in sys.path:
    sys.path.append(dir0)

In [4]:
from mountainsort4_v1 import sort_dataset as ms4_sort_dataset
from summarize_sorting_results import summarize_sorting_results

In [5]:
output_base_dir=os.getcwd()+'/output'
#import shutil
#if os.path.exists(output_base_dir):
#    shutil.rmtree(output_base_dir)
if not os.path.exists(output_base_dir):
    os.mkdir(output_base_dir)

In [6]:
datasets=[]
datasets.append({
    "id":"synth_tetrode_30min",
    "raw_path":"kbucket://b5ecdf1474c5/datasets/synth_datasets/datasets/synth_tetrode_30min"
});
datasets.append({
    "id":"synth_tetrode_120min",
    "raw_path":"kbucket://b5ecdf1474c5/datasets/synth_datasets/datasets/synth_tetrode_120min"
});
#datasets.append({
#    "id":"synth_16ch_30min",
#    "raw_path":"kbucket://b5ecdf1474c5/datasets/synth_datasets/datasets/synth_16ch_30min"
#});

#datasets=[datasets[0]]

In [12]:
verbose='minimal'

def ms4_thr3(dataset_dir,output_dir):
    ms4_sort_dataset(
        dataset_dir=dataset_dir,
        output_dir=output_dir,
        adjacency_radius=-1,
        detect_threshold=3,
        opts={'verbose':verbose}
    )
    
def ms4_thr4(dataset_dir,output_dir):
    ms4_sort_dataset(
        dataset_dir=dataset_dir,
        output_dir=output_dir,
        adjacency_radius=-1,
        detect_threshold=4,
        opts={'verbose':verbose}
    )

algs=[]
algs.append({
    "name":"ms4_thr3",
    "run":ms4_thr3
})
algs.append({
    "name":"ms4_thr4",
    "run":ms4_thr4
})

In [13]:
runs=[]
for i_ds in range(len(datasets)):
    for i_alg in range(len(algs)):
        runs.append({"dataset":datasets[i_ds],"alg":algs[i_alg]})
runs=[runs[0]]

In [14]:
def handle_run(run):
    DS=run['dataset']
    ALG=run['alg']
    print(':::: Applying '+ALG['name']+' to '+DS['id'])
    dsdir=DS['raw_path']
    dsid=DS['id']
    algname=ALG['name']
    output_dir=output_base_dir+'/'+dsid+'--'+algname
    ALG['run'](
        dataset_dir=dsdir,
        output_dir=output_dir
    )
    summarize_sorting_results(
        sorting_output_dir=output_dir,
        output_dir=output_dir+'/summary',
        opts={'verbose':verbose}
    )

In [15]:
from mountainlab_pytools import mlproc as mlp
mlp.initPipeline()
for i in range(len(runs)):
    handle_run(runs[i])
mlp.runPipeline()

:::: Applying ms4_thr3 to synth_tetrode_30min
Running: ml-run-process ephys.bandpass_filter -i timeseries:kbucket://b5ecdf1474c5/datasets/synth_datasets/datasets/synth_tetrode_30min/raw.mda -o timeseries_out:/home/magland/src/mountainsort_examples/spikeforest/output/synth_tetrode_30min--ms4_thr3/filt.mda.prv -p samplerate:30000 freq_max:6000 freq_min:300 --verbose=minimal
JOBS: pending:8 running:1 finished:0
Running: ml-run-process ephys.whiten -i timeseries:/home/magland/src/mountainsort_examples/spikeforest/output/synth_tetrode_30min--ms4_thr3/filt.mda.prv -o timeseries_out:/home/magland/src/mountainsort_examples/spikeforest/output/synth_tetrode_30min--ms4_thr3/pre.mda.prv -p  --verbose=minimal
JOBS: pending:7 running:1 finished:1
Running: ml-run-process ms4alg.sort -i geom:kbucket://b5ecdf1474c5/datasets/synth_datasets/datasets/synth_tetrode_30min/geom.csv timeseries:/home/magland/src/mountainsort_examples/spikeforest/output/synth_tetrode_30min--ms4_thr3/pre.mda.prv -o firings_out:/

In [11]:
type([1,2,3])

list

In [15]:
os.system('ml-run-process ephys.compute_templates -i firings:/home/magland/src/mountainsort_examples/spikeforest/output/synth_tetrode_30min--ms4_thr3/firings.mda timeseries:/home/magland/src/mountainsort_examples/spikeforest/output/synth_tetrode_30min--ms4_thr3/filt.mda -o templates_out:/home/magland/src/mountainsort_examples/spikeforest/output/synth_tetrode_30min--ms4_thr3/summary/templates_filt.mda -p ')

0

In [ ]:
os.system('ml-spec ')